<a href="https://colab.research.google.com/github/samhithasamg/Data-Analysis-and-Visualisation-on-bank-customer-data/blob/main/ProjectScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import gmaps
from geopy.geocoders import Nominatim
import random

Random Sampling of data

In [ ]:
n = sum(1 for line in open('yes_bank.csv')) - 1
s = 100000 #desired sample size
skip = sorted(random.sample(range(1,n+1),n-s))
df= pd.read_csv('yes_bank.csv',skiprows=skip,delimiter='|')

Data Cleaning

In [ ]:
df=df[df['current_address3']!='add3']
df=df[df['current_city']!='addcity']
df=df[df['current_state']!='addstate']
df=df[df['current_pin_code']!='addpin']
df=df[df['current_address3']!='DUMMY']
df=df[df['current_city']!='DUMMY']
df=df[df['current_state']!='DUMMY']
#We have neglected other junk values as the frequency of very small.

In [ ]:
df.reset_index(inplace=True)

Data augmentation

In [ ]:
df2.fillna('',inplace=True) #Required for concatenating the 3 address fields into 1 field

In [ ]:
df['current']=df['current_address3']+' '+df['current_city']+' '+df['current_state'] #complete current address is added as a new column

In [ ]:
df['permanent']=df['permanent_address3']+' '+df['permanent_city']+' '+df['permanent_state'] #complete permanent address is added as a new column

In [ ]:
#new columns augmented for storing latitude and longitude of current and permanent address
df['latitude_c']=np.nan
df['longitude_c']=np.nan
df['latitude_p']=np.nan
df['longitude_p']=np.nan

Generating latitude-longitude

In [ ]:
geolocator = Nominatim(user_agent="yes bank data",timeout=None)
#For each reord, we first find latitude-longitude using the pincode
#If pincode is not available, we go for the complete address
#After both steps, if geolocater is not able to find latitude-longitude, we drop the row.
for i in range(len(df)):
    if df.at[i,'current_pin_code']!='':
        location=geolocator.geocode(df.at[i,'current_pin_code'])
        if location!=None:
            df.at[i,'latitude_c']=location.latitude
            df.at[i,'longitude_c']=location.longitude
    elif df.at[i,'current']!='' :
        location=geolocator.geocode(df.at[i,'current'])
        if location!=None:
            df.at[i,'latitude_c']=location.latitude
            df.at[i,'longitude_c']=location.longitude
    if df.at[i,'permanent_pin_code']!='':
        location=geolocator.geocode(df.at[i,'permanent_pin_code'])
        if location!=None:
            df.at[i,'latitude_p']=location.latitude
            df.at[i,'longitude_p']=location.longitude
    elif df.at[i,'permanent']!='':
        location=geolocator.geocode(df.at[i,'permanent'])
        if location!=None:
            df.at[i,'latitude_p']=location.latitude
            df.at[i,'longitude_p']=location.longitude
    #print(i)

In [ ]:
df.to_csv('yes_bank_address.csv',index=None) #Storing it into csv for future purpose

Generating heatmap using latitude-longitudes

In [ ]:
gmaps.configure(api_key='key') #Specify the googlemap api key

In [ ]:
#For current addresses
locations = df[['latitude_c', 'longitude_c']].dropna()
fig = gmaps.figure() 
heatmap_layer = gmaps.heatmap_layer(locations)

fig.add_layer(gmaps.heatmap_layer(locations)) 
fig 

In [ ]:
#For permanent addresses
locations = df[['latitude_p', 'longitude_p']].dropna()
fig = gmaps.figure() 
heatmap_layer = gmaps.heatmap_layer(locations)

fig.add_layer(gmaps.heatmap_layer(locations)) 
fig 

Classification of households and business customers

In [ ]:
#Since we are identifying the type of customer by current address value, we need to clean the data according to this column.
df=df[df['current_address3']!=' ']
df=df[df['current_address3']!=np.nan]
df=df[df['current_address3'].str.len()>=2]

In [ ]:
#Remove ambiguous addresses which can not be classified as either household or business customer
df=df[df['current'].str.startswith(('NEAR','NR','BESIDE','NR.','B/H','BEHIND','OPP','OPP.','OPPOSITE','BY','NEXT TO','BEFORE'))==False]

Identifying household customers using keywords

In [ ]:
df_house=df[df['current'].str.contains('SOC|SOCIETY|COLONY|TOWER|BUNGLOW')]

Identifying business customers using keywords

In [ ]:
df_business=df[df['current'].str.contains('private limited|ltd.|shop|school|college|tower|company|shop|office|business|market|clinic|bank|station|stn|policestation|corporation|municipality|institute|store|factory|hotel|restaurant|retail|merchant|sons|cooperative|industrial|mill|dairy',case=False)==True]

#Removing the posibility of company being a landmark
df_business=df_business[df_business['current'].str.contains('near|nr|nr.|beside|b/h|behind|opp|opp.|opposite|next to|before',case=False)==False]

Generating heatmaps for household and business customers

In [ ]:
#For household customers
locations = df_house[['latitude_c', 'longitude_c']].dropna()
fig = gmaps.figure() 
heatmap_layer = gmaps.heatmap_layer(locations)

fig.add_layer(gmaps.heatmap_layer(locations)) 
fig 

In [ ]:
#For business customers
locations = df_business[['latitude_c', 'longitude_c']].dropna()
fig = gmaps.figure() 
heatmap_layer = gmaps.heatmap_layer(locations)

fig.add_layer(gmaps.heatmap_layer(locations)) 
fig 